# KG3D TorchicTab Demo 

This demo...

### Visualize Dataset

In [3]:
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
# Open aircraft csv 
aircraft_file_path = 'sfo-aircraft-tail-numbers-and-models.csv'
lines_to_keep = 100
df = pd.read_csv(aircraft_file_path)
df = df.drop(df.columns[0], axis=1).head(lines_to_keep)[["aircraft_model", "airline", "tail_number"]]
df = df.replace(',', '', regex=True)

aircraft_file_path_simple = 'aircraft_models.csv'
df.to_csv(aircraft_file_path_simple, index=False)
df


,aircraft_model,airline,tail_number
0,CRJ-700,Skywest Airlines,N637CA
1,E175-,Skywest Airlines,N205SY
2,B747-400,KLM Royal Dutch Airlines,PHBFB
3,B737-700,Southwest Airlines,N968WN
4,A320-200,United Airlines,N424UA
...,...,...,...
95,B747-400,Qantas Airways,VH-OED
96,B767-200,Amerijet,N743AX
97,EMB-120,Skywest Airlines,N560SW
98,CRJ-200,Skywest Airlines,N412SW


### Produce Semantic Annotations with TorchicTab

In [5]:
from torchic_tab_heuristic.task_calculation_wd import generate_annotations #Tool is still closed-source

(subject_column, 
 primary_annotations, 
 secondary_annotations, 
 cea, 
 cpa, 
 cta) = generate_annotations(aircraft_file_path_simple)

TorchicTab Initiated!
Connecting with Elasticsearch...
Elasticsearch connection succesful!
Loading table...
Table: aircraft_models.csv 
    aircraft_model                   airline tail_number
0         CRJ-700          Skywest Airlines      N637CA
1           E175-          Skywest Airlines      N205SY
2        B747-400  KLM Royal Dutch Airlines       PHBFB
3        B737-700        Southwest Airlines      N968WN
4        A320-200           United Airlines      N424UA
..            ...                       ...         ...
95       B747-400            Qantas Airways      VH-OED
96       B767-200                  Amerijet      N743AX
97        EMB-120          Skywest Airlines      N560SW
98        CRJ-200          Skywest Airlines      N412SW
99       A350-900            Cathay Pacific        BLRM

[100 rows x 3 columns] 

Calculating structural annotations...
Calculating semantic annotations...
Exporting all annotations...


In [6]:
# Get subject column of table
print(f"The subject column of the table is column: \n {df.iloc[:, subject_column]}")

The subject column of the table is column: 
 0     N637CA
1     N205SY
2      PHBFB
3     N968WN
4     N424UA
       ...  
95    VH-OED
96    N743AX
97    N560SW
98    N412SW
99      BLRM
Name: tail_number, Length: 100, dtype: object


In [7]:
# Get generic semantic annotations of table
print("Generic column annotations:")
for i in range(len(df.columns)):
    print("\n",df.columns[i])
    print(df.iloc[:, i])
    print(f"Generic semantic type: {secondary_annotations[i]}")

Generic column annotations:

 aircraft_model
0      CRJ-700
1        E175-
2     B747-400
3     B737-700
4     A320-200
        ...   
95    B747-400
96    B767-200
97     EMB-120
98     CRJ-200
99    A350-900
Name: aircraft_model, Length: 100, dtype: object
Generic semantic type: NE

 airline
0             Skywest Airlines
1             Skywest Airlines
2     KLM Royal Dutch Airlines
3           Southwest Airlines
4              United Airlines
                ...           
95              Qantas Airways
96                    Amerijet
97            Skywest Airlines
98            Skywest Airlines
99              Cathay Pacific
Name: airline, Length: 100, dtype: object
Generic semantic type: ORG

 tail_number
0     N637CA
1     N205SY
2      PHBFB
3     N968WN
4     N424UA
       ...  
95    VH-OED
96    N743AX
97    N560SW
98    N412SW
99      BLRM
Name: tail_number, Length: 100, dtype: object
Generic semantic type: NE


In [8]:
# Get column type annotations
print("Column type annotations:")
for type_annotation in cta:
    column = int(type_annotation[0])
    annotation = type_annotation[1]
    print("\n",df.columns[column])
    print(df.iloc[:, column])
    print(f"Generic semantic type: {secondary_annotations[column]}")
    print(f"Wikidata retrieved annotation: {annotation}")

Column type annotations:

 aircraft_model
0      CRJ-700
1        E175-
2     B747-400
3     B737-700
4     A320-200
        ...   
95    B747-400
96    B767-200
97     EMB-120
98     CRJ-200
99    A350-900
Name: aircraft_model, Length: 100, dtype: object
Generic semantic type: NE
Wikidata retrieved annotation: http://www.wikidata.org/entity/Q15056993

 airline
0             Skywest Airlines
1             Skywest Airlines
2     KLM Royal Dutch Airlines
3           Southwest Airlines
4              United Airlines
                ...           
95              Qantas Airways
96                    Amerijet
97            Skywest Airlines
98            Skywest Airlines
99              Cathay Pacific
Name: airline, Length: 100, dtype: object
Generic semantic type: ORG
Wikidata retrieved annotation: http://www.wikidata.org/entity/Q46970

 tail_number
0     N637CA
1     N205SY
2      PHBFB
3     N968WN
4     N424UA
       ...  
95    VH-OED
96    N743AX
97    N560SW
98    N412SW
99      BLRM


In [9]:
# Get column relationship annotations
print("Column relationships:")
for rel_annotation in cpa:
    if rel_annotation[2][-3:] == "/P1": continue #unkown relationship
    col1 = int(rel_annotation[0])
    col2 = int(rel_annotation[1])
    relationship = rel_annotation[2]
    print(f"\nColumn: {df.columns[col1]} --> {relationship} --> Column: {df.columns[col2]}")
    

Column relationships:

Column: tail_number --> http://www.wikidata.org/prop/direct/P137 --> Column: airline


In [10]:
# Get cell annotations
print("Cell entity annotations:")
for cell_annotation in cea:
    col = int(cell_annotation[0])
    row = int(cell_annotation[1])
    annotation = cell_annotation[2]
    print(f"\nColumn: {df.columns[col]}")
    print(f"Cell name: '{df.iloc[row, col]}' --> Annotation: {annotation}")


Cell entity annotations:

Column: aircraft_model
Cell name: 'CRJ-700' --> Annotation: http://www.wikidata.org/entity/Q962677

Column: aircraft_model
Cell name: 'E175-' --> Annotation: http://www.wikidata.org/entity/Q5322348

Column: aircraft_model
Cell name: 'B747-400' --> Annotation: http://www.wikidata.org/entity/Q906937

Column: aircraft_model
Cell name: 'B737-700' --> Annotation: http://www.wikidata.org/entity/Q15623371

Column: aircraft_model
Cell name: 'A320-200' --> Annotation: http://www.wikidata.org/entity/Q15629179

Column: aircraft_model
Cell name: 'SHORTS-360' --> Annotation: http://www.wikidata.org/entity/Q2140324

Column: aircraft_model
Cell name: 'B737-800' --> Annotation: http://www.wikidata.org/entity/Q80545428

Column: aircraft_model
Cell name: 'EMB-120' --> Annotation: http://www.wikidata.org/entity/Q1163407

Column: aircraft_model
Cell name: 'B747-400' --> Annotation: http://www.wikidata.org/entity/Q906937

Column: aircraft_model
Cell name: 'CRJ-700' --> Annotation:

### Enrich Table Leveraging Semantic Available Semantic Information

Since we have now matched airplanes and airlines with a large external knowledge base such as Wikidata,         
we can query this knowledge base to find out more information about our data and enrich them. 

Wikidata plane attributes that maybe useful to include:    
- Plane first flight (Wikidata property P606)
- Plane maximum operating altitude (Wikidata property P2254)
- PLane height (Wikidata property P2048)
- Plane length (Wikidata property P2043)

In [17]:
plane_column = 0
plane_entities = []
counter = 0
for cell_annotation in cea:
    
    col = int(cell_annotation[0])
    row = int(cell_annotation[1])
    annotation = cell_annotation[2]
    if col == 0: 
        counter += 1
        plane_entities.append(annotation)

plane_entities


['http://www.wikidata.org/entity/Q962677',
 'http://www.wikidata.org/entity/Q5322348',
 'http://www.wikidata.org/entity/Q906937',
 'http://www.wikidata.org/entity/Q15623371',
 'http://www.wikidata.org/entity/Q15629179',
 'http://www.wikidata.org/entity/Q2140324',
 'http://www.wikidata.org/entity/Q80545428',
 'http://www.wikidata.org/entity/Q1163407',
 'http://www.wikidata.org/entity/Q906937',
 'http://www.wikidata.org/entity/Q962677',
 'http://www.wikidata.org/entity/Q891786',
 'http://www.wikidata.org/entity/Q891786',
 'http://www.wikidata.org/entity/Q6484',
 'http://www.wikidata.org/entity/Q1163407',
 'http://www.wikidata.org/entity/Q15634338',
 'http://www.wikidata.org/entity/Q15629179',
 'http://www.wikidata.org/entity/Q31190031',
 'http://www.wikidata.org/entity/Q1163407',
 'http://www.wikidata.org/entity/Q15630770',
 'http://www.wikidata.org/entity/Q15624006',
 'http://www.wikidata.org/entity/Q15623371',
 'http://www.wikidata.org/entity/Q15111575',
 'http://www.wikidata.org/entit

In [18]:
new_plane_properties = ["P606", "P2254", "P2048", "P2043"]
new_plane_property_labels = ["first_flight", "maximum_operating_altitude", "height", "length"]
new_attributes = {property: [] for property in new_plane_property_labels}

In [19]:
#Create the corresponding  SPARQL query 
from SPARQLWrapper import SPARQLWrapper, JSON

def get_wikidata_object(entity, property):
    #Wikidata connection setup
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)
    sparql.addCustomHttpHeader("User-Agent", "MyPythonApp/1.0 (ioannis.dasoulas@kuleuven.be)")

    #Query with a subject entity and object as inputs
    query = f"""
        SELECT ?object WHERE {{
            <{entity}> wdt:{property} ?object .
        }} LIMIT 1
        """
    
    # Query execution
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Results retrieval
    response = sparql.query().convert()
    try:
        results = response['results']['bindings']
        if results:
            return results[0]['object']['value']
        else:
            return None
    except Exception as e:
        print(f"Error querying {entity} for {property}: {e}")
        return None



In [20]:
# Extract new information with the constructed query 
print("Extracting new information for aircraft models...")
for i in range(len(plane_entities)): 
    for j in range(len(new_plane_properties)):
        print(f"\nPlane: {df['aircraft_model'].to_list()[i]}")
        print(f"Extracting {new_plane_property_labels[j].replace('_',' ')} data...")

        if plane_entities[i] is None: 
            print(f"No {new_plane_property_labels[j].replace('_',' ')} data found.")
            new_attributes[new_plane_property_labels[j]].append(None)
            continue

        object = get_wikidata_object(plane_entities[i], new_plane_properties[j])
        if object is None:
            print(f"No {new_plane_property_labels[j].replace('_',' ')} data found.")
            new_attributes[new_plane_property_labels[j]].append(None)
        else:
            print(f"{new_plane_property_labels[j].replace('_',' ')}: {object}")
            new_attributes[new_plane_property_labels[j]].append(object)

Extracting new information for aircraft models...

Plane: CRJ-700
Extracting first flight data...
first flight: 1999-05-27T00:00:00Z

Plane: CRJ-700
Extracting maximum operating altitude data...
maximum operating altitude: 12500

Plane: CRJ-700
Extracting height data...
height: 7.55

Plane: CRJ-700
Extracting length data...
No length data found.

Plane: E175-
Extracting first flight data...
No first flight data found.

Plane: E175-
Extracting maximum operating altitude data...
No maximum operating altitude data found.

Plane: E175-
Extracting height data...
No height data found.

Plane: E175-
Extracting length data...
No length data found.

Plane: B747-400
Extracting first flight data...
first flight: 1988-04-29T00:00:00Z

Plane: B747-400
Extracting maximum operating altitude data...
No maximum operating altitude data found.

Plane: B747-400
Extracting height data...
No height data found.

Plane: B747-400
Extracting length data...
length: 70.66

Plane: B737-700
Extracting first flight 

In [21]:
new_attributes

{'first_flight': ['1999-05-27T00:00:00Z',
  None,
  '1988-04-29T00:00:00Z',
  None,
  None,
  '1981-06-01T00:00:00Z',
  None,
  '1983-07-27T00:00:00Z',
  '1988-04-29T00:00:00Z',
  '1999-05-27T00:00:00Z',
  '1991-05-10T00:00:00Z',
  '1991-05-10T00:00:00Z',
  '2013-06-14T00:00:00Z',
  '1983-07-27T00:00:00Z',
  '1974-12-25T00:00:00Z',
  None,
  None,
  '1983-07-27T00:00:00Z',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  '1991-05-10T00:00:00Z',
  '1999-05-27T00:00:00Z',
  None,
  None,
  None,
  '1991-05-10T00:00:00Z',
  '1988-04-29T00:00:00Z',
  '2013-06-14T00:00:00Z',
  None,
  None,
  None,
  '1988-04-29T00:00:00Z',
  None,
  None,
  None,
  None,
  None,
  '2013-06-14T00:00:00Z',
  None,
  None,
  None,
  None,
  '2013-06-14T00:00:00Z',
  None,
  None,
  '1988-04-29T00:00:00Z',
  '1988-04-29T00:00:00Z',
  None,
  '1983-07-27T00:00:00Z',
  '1991-05-10T00:00:00Z',
  None,
  '2013-06-14T00:00:00Z',
  '1995-08-25T00:00:00Z',
  None,
  '1991-05-10T00:00:00Z',
  None,
  None,
  

In [22]:
#Append discovered attributes to our original CSV
for attribute in new_attributes: 
    df[attribute] = new_attributes[attribute]
df.to_csv(aircraft_file_path_simple, index=False)
df

,aircraft_model,airline,tail_number,first_flight,maximum_operating_altitude,height,length
0,CRJ-700,Skywest Airlines,N637CA,1999-05-27T00:00:00Z,12500,7.55,None
1,E175-,Skywest Airlines,N205SY,None,None,None,None
2,B747-400,KLM Royal Dutch Airlines,PHBFB,1988-04-29T00:00:00Z,None,None,70.66
3,B737-700,Southwest Airlines,N968WN,None,None,None,None
4,A320-200,United Airlines,N424UA,None,None,None,None
...,...,...,...,...,...,...,...
95,B747-400,Qantas Airways,VH-OED,1988-04-29T00:00:00Z,None,None,70.66
96,B767-200,Amerijet,N743AX,None,None,None,None
97,EMB-120,Skywest Airlines,N560SW,1983-07-27T00:00:00Z,9085,None,20
98,CRJ-200,Skywest Airlines,N412SW,1991-05-10T00:00:00Z,None,None,None


### AutoRML

Automatic construction of a knowledge graph from the CSV using the semantic annotations discovered. 